In [ ]:
import numpy as np

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

import torch # For building the networks
import torchtuples as tt # Some useful functions

from pycox.datasets import metabric
from pycox.models import LogisticHazard

from Evaluations.One_Calibration import one_calibration_pycox
from Evaluations.BrierScore import single_brier_score_pycox, integrated_brier_score_pycox
from Evaluations.Concordance import concordance_pycox
from Evaluations.L1 import l1_loss_pycox
from Evaluations.D_Calibration import d_calibration_pycox

from Evaluator import PycoxEvaluator

In [ ]:
np.random.seed(1234)
_ = torch.manual_seed(123)

df_train = metabric.read_df()
df_test = df_train.sample(frac=0.2)
df_train = df_train.drop(df_test.index)
df_val = df_train.sample(frac=0.2)
df_train = df_train.drop(df_val.index)

cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
cols_leave = ['x4', 'x5', 'x6', 'x7']

standardize = [([col], StandardScaler()) for col in cols_standardize]
leave = [(col, None) for col in cols_leave]

x_mapper = DataFrameMapper(standardize + leave)

x_train = x_mapper.fit_transform(df_train).astype('float32')
x_val = x_mapper.transform(df_val).astype('float32')
x_test = x_mapper.transform(df_test).astype('float32')


num_durations = 10

labtrans = LogisticHazard.label_transform(num_durations)
# labtrans = PMF.label_transform(num_durations)
# labtrans = DeepHitSingle.label_transform(num_durations)

get_target = lambda df: (df['duration'].values, df['event'].values)
y_train = labtrans.fit_transform(*get_target(df_train))
y_val = labtrans.transform(*get_target(df_val))

train = (x_train, y_train)
val = (x_val, y_val)

# We don't need to transform the test labels
durations_test, events_test = get_target(df_test)
durations_train, events_train = get_target(df_train)

in_features = x_train.shape[1]
num_nodes = [32, 32]
out_features = labtrans.out_features
batch_norm = True
dropout = 0.1

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)

model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts)

batch_size = 256
epochs = 100
callbacks = [tt.cb.EarlyStopping()]

log = model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=val)

surv = model.interpolate(10).predict_surv_df(x_test)

In [ ]:
eval = PycoxEvaluator(surv, durations_test, events_test, durations_train, events_train)

In [ ]:
cindex = eval.concordance()

In [ ]:
bs = eval.brier_score()
ibs2 = eval.integrated_brier_score()

In [ ]:
t_time = round(np.percentile(durations_test, 50))
p_value1, ob1, pre1 = one_calibration_pycox(surv, durations_test, event_indicator=events_test, target_time=t_time, method="DN")
p_value2, ob2, pre2 = eval.one_calibration(target_time=t_time, method="DN")

In [ ]:
l11 = l1_loss_pycox(surv, durations_test, events_test, durations_train, events_train, method='Margin', predicted_time_method="Mean")
l12 = eval.l1_loss(method='Margin', predicted_time_method="Mean")
print("l1 loss: {} and {}".format(l11, l12))
p_value1 = d_calibration_pycox(surv, durations_test, events_test)
p_value2 = eval.d_calibration()
print("D-cal: {} and {}".format(p_value1, p_value2))

# ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
# ev.concordance_td('antolini')
# time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
# ev.integrated_brier_score(time_grid)
